In [1]:
import clip
import argparse
import numpy as np

from loaders import CxrDataLoader
from datasets import _build_iu_xray_sampler
from classifier.model import ChexpertXRClassifier
from classifier.trainer import ChexpertClassifierTrainer

In [2]:
LABELS = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
          'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax',
          'Support Devices']

args = argparse.Namespace(
    dataset_name='iu-xray',
    batch_size=16,
    num_workers=0,
    max_seq_length=248
)

In [3]:
clip_model, preprocess = clip.load("ViT-B/32", load_from_clip=True)
clip_model.eval()

vision_model = clip_model.visual
input_dim = clip_model.vision_width
hidden_dim = clip_model.transformer_width

model = ChexpertXRClassifier(vision_model, input_dim, hidden_dim, num_classes=len(LABELS))

In [4]:
sampler = _build_iu_xray_sampler('train')

In [5]:
dataloaders = {
    'train': CxrDataLoader(args, split='train', transform=preprocess, sampler=sampler),
    'val': CxrDataLoader(args, split='val', transform=preprocess),
    'test': CxrDataLoader(args, split='test', transform=preprocess)
}

In [6]:
trainer = ChexpertClassifierTrainer(model, train_loader=dataloaders['train'], val_loader=dataloaders['val'], log_interval=10)

2025-04-05 22:51:13,392 - INFO - Total parameters: 88,250,126
2025-04-05 22:51:13,393 - INFO - Trainable parameters: 88,250,126 (100.00%)


In [7]:
trainer.train(epochs=10)

2025-04-05 22:51:13,460 - INFO - Start training
2025-04-05 22:51:13,462 - INFO - Starting epoch 1/10
100%|██████████| 169/169 [03:35<00:00,  1.27s/it]
2025-04-05 22:54:48,781 - INFO - Epoch 1 completed in 215.32s. Average loss: 0.3546
2025-04-05 22:54:48,782 - INFO - Running evaluation on val set
Val Evaluation: 100%|██████████| 37/37 [00:31<00:00,  1.19it/s]
2025-04-05 22:55:19,822 - INFO - Number of uncertain cases predicted near 0.5: 0
2025-04-05 22:55:19,856 - INFO - Val metrics: {
  "val_loss": 0.22427096238007416,
  "val_auc": 0.5323510433284181
}
2025-04-05 22:55:19,856 - INFO - Starting epoch 2/10
100%|██████████| 169/169 [03:35<00:00,  1.28s/it]
2025-04-05 22:58:55,377 - INFO - Epoch 2 completed in 215.51s. Average loss: 0.3411
2025-04-05 22:58:55,377 - INFO - Running evaluation on val set
Val Evaluation: 100%|██████████| 37/37 [00:30<00:00,  1.21it/s]
2025-04-05 22:59:26,007 - INFO - Number of uncertain cases predicted near 0.5: 60
2025-04-05 22:59:26,040 - INFO - Val metrics

KeyboardInterrupt: 

In [8]:
test_loader = dataloaders['test']
test_loader_iter = iter(test_loader)
batch = next(test_loader_iter)

labels = batch['labels'][0]
labels[labels == -1] = 0.0
image = batch['image'][0]
image = image.unsqueeze(0)
predictions = model(image)

print(predictions['probs'])
print(labels)

tensor([[0.0911, 0.2139, 0.0697, 0.0712, 0.0050, 0.1004, 0.0641, 0.3036, 0.0780,
         0.1526, 0.0532, 0.0622, 0.1173, 0.1548]], grad_fn=<SigmoidBackward0>)
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
